#Install lib

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00


In [ ]:
#!apt install git-lfs

#Enviroment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible answers are allowed or not).
squad_v2 = True
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = 'twmkn9/distilbert-base-uncased-squad2'
batch_size = 16

#Preprocessing the training data

In [ ]:
import json
from pathlib import Path
import torch

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def read_squad_train(path,data_index,context_size):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  size=0
  for group in squad_dict['data'][data_index-1:data_index]:
      for passage in group['paragraphs']:
        if size <= context_size:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                if answer['answer_start']>=0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
          size+=1
        else: break

  return contexts, questions, answers
def read_squad_val(path):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)

  return contexts, questions, answers
def add_token_positions(encodings, answers):
  """
  Convert answer start- end from string to token
  """
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
        start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
        end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
def add_end_idx(answers, contexts):
  """
  Squad format chỉ có id start của câu trả lời
  -> Hàm này dùng để tạo id end của câu trả lời
  """
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two – fix this
    if context[start_idx:end_idx] == gold_text:
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

train_contexts, train_questions, train_answers = read_squad_train('/content/drive/MyDrive/DoAnCK/Dataset/squad_vi.json', 1,3500)

add_end_idx(train_answers, train_contexts)
 

#train
new_train_contexts=[]
new_train_questions=[]
new_train_answers=[]
for i in range(len(train_answers)):
  if train_answers[i]['answer_start']!=train_answers[i]['answer_end']:
    new_train_contexts.append(train_contexts[i])
    new_train_questions.append(train_questions[i])
    new_train_answers.append(train_answers[i])
(train_contexts, train_questions, train_answers)=(new_train_contexts, new_train_questions, new_train_answers)
(new_train_contexts, new_train_questions, new_train_answers)=(0,0,0)

#chia train val
val_contexts, val_questions, val_answers = train_contexts[12000:], train_questions[12000:], train_answers[12000:]
train_contexts, train_questions, train_answers = train_contexts[:12000], train_questions[:12000], train_answers[:12000]


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)



Tải data lên pytorch

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

#Load pretrain

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

#Train

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [ ]:
epochs = 10
batch_size=8
train_count = len(train_dataset)
val_count = len(val_dataset)
# PATH = "/content/drive/MyDrive/CS221/DoAnCK/Model/best_model.model"

###Train with trainer

###10 epoch

In [ ]:
import numpy as np

In [ ]:
args = TrainingArguments(
    f"/content/drive/MyDrive/LOG",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01
    #push_to_hub=True,
)

In [ ]:
def f1_score(start_pred,end_pred,start_true,end_true):
  a=np.arange(int(start_pred),int(end_pred+1), step=1)
  b=np.arange(int(start_true),int(end_true+1), step=1)
  true_count = len(np.intersect1d(a,b))
  if ((int(end_true)-int(start_true)) == 0):
    recall = 0
  else:
    recall = true_count/(int(end_true)-int(start_true))
  if ((int(end_pred)-int(start_pred)) == 0):
    precision = 0
  else:
    precision= true_count/(int(end_pred)-int(start_pred))
  if (precision + recall == 0):
    return 0
  f1=2*(precision*recall)/(precision+recall)
  return f1

In [ ]:
def compute_metrics(eval_pred):    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    count = 0
    for i in range(len(predictions[0])):
      if (predictions[0][i] == labels[0][i] and predictions[1][i] == labels[1][i]):
        count += 1
    f1 = 0
    for i in range(len(predictions[0])):
      f1 += f1_score(predictions[0][i], predictions[1][i], labels[0][i], labels[1][i])

    return {'exact_match': round((count/len(predictions[0]))*100, 3), 'f1_score': round(f1/len(predictions[0]),3)} #metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000
  Number of trainable parameters = 66364418
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,3.147900,2.793257,20.853000,0.340000
2,2.475400,2.564300,26.143000,0.420000
3,2.067000,2.467136,29.789000,0.465000
4,1.726700,2.530016,30.457000,0.474000
5,1.483700,2.666125,31.998000,0.494000
6,1.251200,2.707475,31.279000,0.500000
7,1.058300,3.067852,31.433000,0.505000
8,0.904700,3.105921,31.073000,0.497000
9,0.821000,3.253505,31.073000,0.499000
10,0.709700,3.402500,31.022000,0.499000


***** Running Evaluation *****
  Num examples = 1947
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/checkpoint-1500
Configuration saved in /content/drive/MyDrive/LOG/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/LOG/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/checkpoint-1500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/checkpoint-1500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1947
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/LOG/checkpoint-3000
Configuration saved in /content/drive/MyDrive/LOG/checkpoint-3000/config.json
Model weights saved in /content/drive/MyDrive/LOG/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/LOG/checkpoint-3000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/LOG/checkpoint-3000/special_tokens_map.json
***** Runn

TrainOutput(global_step=15000, training_loss=1.5991810811360676, metrics={'train_runtime': 6121.559, 'train_samples_per_second': 19.603, 'train_steps_per_second': 2.45, 'total_flos': 1.567837200384e+16, 'train_loss': 1.5991810811360676, 'epoch': 10.0})

In [ ]:
test_contexts, test_questions, test_answers = read_squad_val('/content/drive/MyDrive/DoAnCK/Dataset/Mailong25/train-v2.0.json')
add_end_idx(test_answers, test_contexts)
new_test_contexts=[]
new_test_questions=[]
new_test_answers=[]
for i in range(len(test_answers)):
  if test_answers[i]['answer_start']!=test_answers[i]['answer_end']:
    new_test_contexts.append(test_contexts[i])
    new_test_questions.append(test_questions[i])
    new_test_answers.append(test_answers[i])
(test_contexts, test_questions, test_answers)=(new_test_contexts, new_test_questions, new_test_answers)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)
add_token_positions(test_encodings, test_answers)
test_dataset = SquadDataset(test_encodings)

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model.load_state_dict(torch.load('/content/drive/MyDrive/LOG/best/pytorch_model.bin'))
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--twmkn9--distilbert-base-uncased-squad2/snapshots/84a8ab9f335a39297b69790f023f9e5ff70a3734/config.json
Model config DistilBertConfig {
  "_name_or_path": "twmkn9/distilbert-base-uncased-squad2",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "eos_token_ids": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--twmkn9--distilbert-base-uncased-squad2/snapshots/84a8ab9f335a39297b69790f023f9e5ff70a3734/py

{'eval_loss': 3.6467223167419434,
 'eval_exact_match': 19.594,
 'eval_f1_score': 0.489,
 'eval_runtime': 14.1848,
 'eval_samples_per_second': 69.44,
 'eval_steps_per_second': 4.371}

Evaluation